In [ ]:
import torch
import random
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Standard library
import json
import os
import random
import time

# Third-party libraries
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import torch

# Scikit-learn - core modules
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Scikit-learn - metrics
from sklearn.metrics import (
    accuracy_score, average_precision_score, balanced_accuracy_score,
    ConfusionMatrixDisplay, f1_score, log_loss,
    matthews_corrcoef, mean_squared_error, precision_score,
    PrecisionRecallDisplay, r2_score, recall_score, roc_auc_score, RocCurveDisplay
)

# Local application/library imports
from utils import load_search_space, get_model


## DATASET

In [ ]:
SEED = 64

# Set random seeds
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Dataset Info
# adult_income_cleaned, framingham_cleaned, preprocessed_heloc, diabetes
dataset_name = 'boston'        
dataset_subpath = 'Regression/boston'       
task_type = 'Regression'

In [ ]:
# Dataset Info
# adult_income_cleaned, framingham_cleaned, preprocessed_heloc, diabetes
dataset_name = 'cmc'        
dataset_subpath = 'Multiclass/cmc'       
task_type = 'Multiclass'

In [ ]:
# Dataset Info
# adult_income_cleaned, framingham_cleaned, preprocessed_heloc, diabetes
dataset_name = 'preprocessed_heloc'        
dataset_subpath = 'Binary/heloc'       
task_type = 'Binary'

In [ ]:
df = pd.read_csv(f"./data/{dataset_subpath}/{dataset_name}.csv")

In [ ]:
df.shape

In [ ]:
df.head()

## LOAD AND PREPROCESS

In [ ]:
def preprocess_data(df, dataset_name, task_type, seed=42):
    task_type = task_type.lower()

    # Load config
    with open(f"./configs/preprocess/{dataset_name}.json") as f:
        config = json.load(f)

    categorical_cols = config["categorical_cols"]
    numerical_cols = config["numerical_cols"]
    encoding = config["encoding"]

    # Extract features and target
    X = df[numerical_cols + categorical_cols].copy()
    y = df.iloc[:, -1].copy()

    # Encode target if needed
    le = None
    if encoding.get("target") == "label":
        le = LabelEncoder()
        y = le.fit_transform(y)
        label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    else:
        label_mapping = None

    # Split raw data before transformation
    if task_type == "regression":
        # For regression, we can use a simple split
        X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=seed
        )
        X_val_raw, X_test_raw, y_val, y_test = train_test_split(
            X_temp_raw, y_temp, test_size=0.5, random_state=seed
        )
    else:
        # For classification, we need stratified splits
        X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=seed, stratify=y
        )
        X_val_raw, X_test_raw, y_val, y_test = train_test_split(
            X_temp_raw, y_temp, test_size=0.5, random_state=seed, stratify=y_temp
        )

    # Ensure y_* are Series with index matching the X_*
    y_train = pd.Series(y_train, index=X_train_raw.index)
    y_val = pd.Series(y_val, index=X_val_raw.index)
    y_test = pd.Series(y_test, index=X_test_raw.index)

    # Compute class weights for classification
    class_weight = None
    if task_type in ["binary", "multiclass"]:
        class_weight_values = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
        class_weight = dict(zip(np.unique(y_train), class_weight_values))
        print(f"Class weights: {class_weight}")

    # Transform numerical and categorical features
    transformers = []

    if encoding["numerical_features"] == "minmax":
        transformers.append(("num", MinMaxScaler(), numerical_cols))
    elif encoding["numerical_features"] == "standard":
        transformers.append(("num", StandardScaler(), numerical_cols))

    if categorical_cols and encoding["categorical_features"] == "onehot":
        transformers.append(("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_cols))

    if transformers:
        preprocessor = ColumnTransformer(transformers=transformers)
        X_train = preprocessor.fit_transform(X_train_raw)
        X_val = preprocessor.transform(X_val_raw)
        X_test = preprocessor.transform(X_test_raw)

        # Recover transformed column names
        if "cat" in preprocessor.named_transformers_:
            cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols)
            all_feature_names = numerical_cols + list(cat_feature_names)
        else:
            all_feature_names = numerical_cols + categorical_cols

        X_train = pd.DataFrame(X_train, columns=all_feature_names, index=X_train_raw.index)
        X_val = pd.DataFrame(X_val, columns=all_feature_names, index=X_val_raw.index)
        X_test = pd.DataFrame(X_test, columns=all_feature_names, index=X_test_raw.index)
    else:
        all_feature_names = numerical_cols + categorical_cols  # or keep original order
        X_train = pd.DataFrame(X_train_raw, columns=all_feature_names, index=X_train_raw.index)
        X_val = pd.DataFrame(X_val_raw, columns=all_feature_names, index=X_val_raw.index)
        X_test = pd.DataFrame(X_test_raw, columns=all_feature_names, index=X_test_raw.index)

    print(f"Shapes — Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print(f"Numerical features: {len(numerical_cols)} — {numerical_cols}")
    print(f"Categorical features: {len(categorical_cols)} — {categorical_cols}")
    print(f"Total features: {X_train.shape[1]}")
    if label_mapping:
        print(f"Target label mapping: {label_mapping}")

    return (
        X_train, X_val, X_test,
        y_train, y_val, y_test,
        None, le, class_weight
    )


## Generate Images

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, categorical_cols, label_encoder, class_weight = preprocess_data(df, dataset_name=dataset_name, task_type=task_type, seed=SEED)

In [ ]:
# 2. Concatenate X and y (label as last column)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(f"Shapes — Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

In [ ]:
from generate_images import generate_images_from_config, load_config

In [ ]:
config_path = f"configs/image_generation/{dataset_name}.json"
config = load_config(config_path)

generate_images_from_config(
    config=config,
    X_train=train_df,
    X_val=val_df,
    X_test=test_df,
)

In [ ]:
from pathlib import Path
import os
from PIL import Image

# Base folder containing train/val/test
BASE = Path(f"SyntheticImages/{task_type}/{dataset_name}/BIE")
SPLITS = ["train", "val", "test"]
SUB = "images"

# Padding background color
RGB_PAD_COLOR = (0, 0, 0)  # white

def pad_to_square_right_or_bottom(im: Image.Image) -> Image.Image:
    """Pad image to square by extending to the right (if wider) or bottom (if taller) with black background."""
    w, h = im.size
    if w == h:
        return im  # already square

    size = max(w, h)

    # Ensure RGB mode
    if im.mode != "RGB":
        im = im.convert("RGB")

    new_im = Image.new("RGB", (size, size), RGB_PAD_COLOR)

    if w > h:
        # wider → paste at top-left, pad bottom
        new_im.paste(im, (0, 0))
    else:
        # taller → paste at top-left, pad right
        new_im.paste(im, (0, 0))

    return new_im

def is_image(fname: str) -> bool:
    return fname.lower().endswith((".png", ".jpg", ".jpeg"))

# Run the padding
count_total, count_padded, count_skipped = 0, 0, 0
for split in SPLITS:
    img_dir = BASE / split / SUB
    if not img_dir.exists():
        print(f"[warn] missing dir: {img_dir}")
        continue
    for fname in os.listdir(img_dir):
        if not is_image(fname):
            continue
        path = img_dir / fname
        try:
            with Image.open(path) as im:
                count_total += 1
                if im.size[0] == im.size[1]:
                    count_skipped += 1
                    continue
                out = pad_to_square_right_or_bottom(im)
                out.save(path, format=im.format)  # overwrite
                count_padded += 1
        except Exception as e:
            print(f"[error] {path}: {e}")

print(f"\nDone. Total: {count_total}, padded: {count_padded}, already square: {count_skipped}")


In [ ]:
from pathlib import Path
from PIL import Image

# Base folder containing train/val/test
BASE = Path(f"SyntheticImages/{task_type}/{dataset_name}/BIE")
SPLITS = ["train", "val", "test"]

# Padding background color
RGB_PAD_COLOR = (0, 0, 0)  # black


def pad_to_square_right_or_bottom(im: Image.Image) -> Image.Image:
    """Pad image to square by extending to the right (if wider) or bottom (if taller) with black background."""
    w, h = im.size
    if w == h:
        return im  # already square

    size = max(w, h)

    # Ensure RGB mode
    if im.mode != "RGB":
        im = im.convert("RGB")

    new_im = Image.new("RGB", (size, size), RGB_PAD_COLOR)

    if w > h:
        # wider → paste at top-left, pad bottom
        new_im.paste(im, (0, 0))
    else:
        # taller → paste at top-left, pad right
        new_im.paste(im, (0, 0))

    return new_im


def is_image(fname: str) -> bool:
    return fname.lower().endswith((".png", ".jpg", ".jpeg"))


# Run the padding
count_total, count_padded, count_skipped = 0, 0, 0
for split in SPLITS:
    split_dir = BASE / split
    if not split_dir.exists():
        print(f"[warn] missing dir: {split_dir}")
        continue

    # Loop over folders directly inside train/val/test
    for class_dir in split_dir.iterdir():
        if not class_dir.is_dir():
            print(f"[skip non-folder] {class_dir}")
            continue

        for path in class_dir.iterdir():
            if not path.is_file() or not is_image(path.name):
                continue
            try:
                with Image.open(path) as im:
                    count_total += 1
                    if im.size[0] == im.size[1]:
                        count_skipped += 1
                        continue
                    out = pad_to_square_right_or_bottom(im)
                    out.save(path, format=im.format)  # overwrite
                    count_padded += 1
            except Exception as e:
                print(f"[error] {path}: {e}")

print(f"\nDone. Total: {count_total}, padded: {count_padded}, already square: {count_skipped}")


In [ ]:
import math

def suggest_featurewrap_size(
    n_features: int,
    bins: int = 10,
    bits_per_pixel: int = 8,
    safety_factor: float = 1.0,
):
    """
    Suggest minimal (height, width) image size for FeatureWrap so that
    the binary vector fits without raising the 'size too small' error.

    Assumes (approx):
        total_bits ~= n_features * bins

    Parameters
    ----------
    n_features : int
        Number of input features (columns in X).
    bins : int, default=10
        Number of bins used for numeric features in FeatureWrap.
    bits_per_pixel : int, default=8
        Bits aggregated into a single pixel (FeatureWrap uses 8).
    safety_factor : float, default=1.0
        Multiplier to add a margin (e.g. 1.2 for +20% pixels).

    Returns
    -------
    (height, width) : tuple[int, int]
        Suggested square image size.
    """
    if n_features <= 0:
        raise ValueError("n_features must be positive")

    total_bits = n_features * bins
    pixels_needed = math.ceil((total_bits * safety_factor) / bits_per_pixel)
    dim = math.ceil(math.sqrt(pixels_needed))
    return dim, dim

In [ ]:
suggest_featurewrap_size(76)